In [1]:
!pip install elasticsearch==8.11 #Elasticsearch

In [2]:
pip -q install eland elasticsearch sentence_transformers transformers torch==2.1.0

Note: you may need to restart the kernel to use updated packages.


In [3]:
from elasticsearch import Elasticsearch, helpers
from urllib.request import urlopen
import json
from pathlib import Path
from eland.ml.pytorch import PyTorchModel
from eland.ml.pytorch.transformers import TransformerModel
from elasticsearch.client import MlClient
import getpass

/var/folders/5r/774p5f6x3jb5md2ycsn04blc0000gn/T/ipykernel_4821/3060853836.py:7: DeprecationWarning: Importing from the 'elasticsearch.client' module is deprecated. Instead use 'elasticsearch' module for importing the client.
  from elasticsearch.client import MlClient


In [4]:
# Found in the 'Manage Deployment' page
CLOUD_ID = getpass.getpass('Enter Elastic Cloud ID:  ')

# Password for the 'elastic' user generated by Elasticsearch
ELASTIC_PASSWORD = getpass.getpass('Enter Elastic password:  ')

# Create the client instance
client = Elasticsearch(
    cloud_id=CLOUD_ID,
    basic_auth=("elastic", ELASTIC_PASSWORD),
    request_timeout=3600
)

Enter Elastic Cloud ID:   ········
Enter Elastic password:   ········


In [7]:
# Set the model name from Hugging Face and task type
# open ai detector model - developed by open ai https://github.com/openai/gpt-2-output-dataset/tree/master/detector
hf_model_id='roberta-base-openai-detector' #updated
tm = TransformerModel(model_id=hf_model_id, task_type="text_classification") #updated

#set the modelID as it is named in Elasticsearch
es_model_id = tm.elasticsearch_model_id()

# Download the model from Hugging Face
tmp_path = "models"
Path(tmp_path).mkdir(parents=True, exist_ok=True)
model_path, config, vocab_path = tm.save(tmp_path)

# Load the model into Elasticsearch
ptm = PyTorchModel(client, es_model_id)
ptm.import_model(model_path=model_path, config_path=None, vocab_path=vocab_path, config=config)

#Start the model
s = MlClient.start_trained_model_deployment(client, model_id=es_model_id)
s.body

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-base-openai-detector were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
STAGE:2024-01-05 10:28:30 4821:36646 ActivityProfilerController.cpp:312] Completed Stage: Warm Up
STAGE:2024-01-05 10:28:30 4821:36646 ActivityProfilerController.cpp:318] Completed Stage: Collection
STAGE:2024-01-05 10:28:30 4821:36646 ActivityProfilerController.cpp:322] Completed Stage: Post Processing


  0%|          | 0/476 [00:00<?, ? parts/s]

{'assignment': {'task_parameters': {'model_id': 'roberta-base-openai-detector',
   'deployment_id': 'roberta-base-openai-detector',
   'model_bytes': 498663890,
   'threads_per_allocation': 1,
   'number_of_allocations': 1,
   'queue_capacity': 1024,
   'cache_size': '498663890b',
   'priority': 'normal',
   'per_deployment_memory_bytes': 498596904,
   'per_allocation_memory_bytes': 632532156},
  'routing_table': {'2Aa4SRDxTJO2f5b1gRc_4Q': {'current_allocations': 1,
    'target_allocations': 1,
    'routing_state': 'started',
    'reason': ''}},
  'assignment_state': 'started',
  'start_time': '2024-01-05T15:33:56.506211165Z',
  'max_assigned_allocations': 1}}

In [8]:
# Set the model name from Hugging Face and task type
# sentence-transformers model
hf_model_id='sentence-transformers/all-mpnet-base-v2' #updated
tm = TransformerModel(model_id=hf_model_id, task_type="text_embedding") #updated

#set the modelID as it is named in Elasticsearch
es_model_id = tm.elasticsearch_model_id()

# Download the model from Hugging Face
tmp_path = "models"
Path(tmp_path).mkdir(parents=True, exist_ok=True)
model_path, config, vocab_path = tm.save(tmp_path)

# Load the model into Elasticsearch
ptm = PyTorchModel(client, es_model_id)
ptm.import_model(model_path=model_path, config_path=None, vocab_path=vocab_path, config=config)

# Start the model
s = MlClient.start_trained_model_deployment(client, model_id=es_model_id)
s.body

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

STAGE:2024-01-05 10:38:32 4821:36646 ActivityProfilerController.cpp:312] Completed Stage: Warm Up
STAGE:2024-01-05 10:38:32 4821:36646 ActivityProfilerController.cpp:318] Completed Stage: Collection
STAGE:2024-01-05 10:38:32 4821:36646 ActivityProfilerController.cpp:322] Completed Stage: Post Processing


  0%|          | 0/416 [00:00<?, ? parts/s]

{'assignment': {'task_parameters': {'model_id': 'sentence-transformers__all-mpnet-base-v2',
   'deployment_id': 'sentence-transformers__all-mpnet-base-v2',
   'model_bytes': 435655636,
   'threads_per_allocation': 1,
   'number_of_allocations': 1,
   'queue_capacity': 1024,
   'cache_size': '435655636b',
   'priority': 'normal',
   'per_deployment_memory_bytes': 435587600,
   'per_allocation_memory_bytes': 700162236},
  'routing_table': {'2Aa4SRDxTJO2f5b1gRc_4Q': {'current_allocations': 1,
    'target_allocations': 1,
    'routing_state': 'started',
    'reason': ''}},
  'assignment_state': 'started',
  'start_time': '2024-01-05T15:43:19.430653116Z',
  'max_assigned_allocations': 1}}

In [9]:
#source index
client.indices.create(
index="plagiarism-docs",
mappings= {
    "properties": {
        "title": {
            "type": "text",
            "fields": {
                "keyword": {
                "type": "keyword"
                }
            }
        },
        "abstract": {
            "type": "text",
            "fields": {
                "keyword": {
                "type": "keyword"
                }
            }
        },
        "url": {
            "type": "keyword"
        },
        "venue": {
            "type": "keyword"
        },
         "year": {
            "type": "keyword"
        }      
    }
})

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'plagiarism-docs'})

In [10]:
#ingest pipeline

client.ingest.put_pipeline( 
    id="plagiarism-checker-pipeline",
    processors = [
    {
      "inference": { #for ml models - to infer against the data that is being ingested in the pipeline
        "model_id": "roberta-base-openai-detector", #text classification model id
        "target_field": "openai-detector", # Target field for the inference results
        "field_map": { #Maps the document field names to the known field names of the model.
        "abstract": "text_field" # Field matching our configured trained model input. 
        }
      }
    },
    {
      "inference": {
        "model_id": "sentence-transformers__all-mpnet-base-v2", #text embedding model 
        "target_field": "abstract_vector", # Target field for the inference results
        "field_map": {
        "abstract": "text_field" # Field matching our configured trained model input. Typically for NLP models, the field name is text_field.
        }
      }
    }
    
  ]
)

ObjectApiResponse({'acknowledged': True})

In [11]:
client.indices.create(
index="plagiarism-checker",
mappings={
"properties": {
    "title": {
        "type": "text",
        "fields": {
            "keyword": {
                "type": "keyword"
            }
        }
    },
    "abstract": {
        "type": "text",
        "fields": {
            "keyword": {
                "type": "keyword"
            }
        }
    },
    "url": {
        "type": "keyword"
    },
    "venue": {
        "type": "keyword"
    },
    "year": {
        "type": "keyword"
    }, 
    "abstract_vector.predicted_value": { # Inference results field, target_field.predicted_value
    "type": "dense_vector", 
    "dims": 768, # embedding_size
    "index": "true", 
    "similarity": "dot_product" #  When indexing vectors for approximate kNN search, you need to specify the similarity function for comparing the vectors.
         }
  }
}
)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'plagiarism-checker'})

In [13]:
from elasticsearch import Elasticsearch, helpers

In [14]:
url = "https://public.ukp.informatik.tu-darmstadt.de/reimers/sentence-transformers/datasets/emnlp2016-2018.json"

# Send a request to the URL and get the response
response = urlopen(url)

# Load the response data into a JSON object
data_json = json.loads(response.read())

def create_index_body(doc):
    """ Generate the body for an Elasticsearch document. """
    return {
        "_index": "plagiarism-docs",
        "_source": doc,
    }

# Prepare the documents to be indexed
documents = [create_index_body(doc) for doc in data_json]

# Use helpers.bulk to index
helpers.bulk(client, documents)

print("Done indexing documents into `plagiarism-docs` index")

Done indexing documents into `plagiarism-docs` index


In [16]:
client.reindex(wait_for_completion=True,
               source={
                  "index": "plagiarism-docs"
    },
               dest= {
                  "index": "plagiarism-checker",
                  "pipeline": "plagiarism-checker-pipeline"
    }
)

ObjectApiResponse({'took': 26478, 'timed_out': False, 'total': 974, 'updated': 974, 'created': 0, 'deleted': 0, 'batches': 1, 'version_conflicts': 0, 'noops': 0, 'retries': {'bulk': 0, 'search': 0}, 'throttled_millis': 0, 'requests_per_second': -1.0, 'throttled_until_millis': 0, 'failures': []})

In [17]:
#duplicated text - direct plagiarism

model_text = 'Understanding and reasoning about cooking recipes is a fruitful research direction towards enabling machines to interpret procedural text. In this work, we introduce RecipeQA, a dataset for multimodal comprehension of cooking recipes. It comprises of approximately 20K instructional recipes with multiple modalities such as titles, descriptions and aligned set of images. With over 36K automatically generated question-answer pairs, we design a set of comprehension and reasoning tasks that require joint understanding of images and text, capturing the temporal flow of events and making sense of procedural knowledge. Our preliminary results indicate that RecipeQA will serve as a challenging test bed and an ideal benchmark for evaluating machine comprehension systems. The data and leaderboard are available at http://hucvl.github.io/recipeqa.'

response = client.search(index='plagiarism-checker', size=1,
    knn={
        "field": "abstract_vector.predicted_value",
        "k": 9,
        "num_candidates": 974,
        "query_vector_builder": {
            "text_embedding": {
                "model_id": "sentence-transformers__all-mpnet-base-v2",
                "model_text": model_text
            }
        }
    }
)

for hit in response['hits']['hits']:
    score = hit['_score']
    title = hit['_source']['title']
    abstract = hit['_source']['abstract']
    openai = hit['_source']['openai-detector']['predicted_value']
    url = hit['_source']['url']

    if score > 0.9:
        print(f"\nHigh similarity detected! This might be plagiarism.")
        print(f"\nMost similar document: '{title}'\n\nAbstract: {abstract}\n\nurl: {url}\n\nScore:{score}\n")

        if openai == 'Fake':
            print("This document may have been created by AI.\n")

    elif score < 0.7:
        print(f"\nLow similarity detected. This might not be plagiarism.")

        if openai == 'Fake':
            print("This document may have been created by AI.\n")

    else:
        print(f"\nModerate similarity detected.")
        print(f"\nMost similar document: '{title}'\n\nAbstract: {abstract}\n\nurl: {url}\n\nScore:{score}\n")

        if openai == 'Fake':
            print("This document may have been created by AI.\n")

ml_client = MlClient(client)

model_id = 'roberta-base-openai-detector' #open ai text classification model

document = [
    {
        "text_field": model_text
    }
]

ml_response = ml_client.infer_trained_model(model_id=model_id, docs=document)

predicted_value = ml_response['inference_results'][0]['predicted_value']

if predicted_value == 'Fake':
    print("Note: The text query you entered may have been generated by AI.\n")



High similarity detected! This might be plagiarism.

Most similar document: 'RecipeQA: A Challenge Dataset for Multimodal Comprehension of Cooking Recipes'

Abstract: Understanding and reasoning about cooking recipes is a fruitful research direction towards enabling machines to interpret procedural text. In this work, we introduce RecipeQA, a dataset for multimodal comprehension of cooking recipes. It comprises of approximately 20K instructional recipes with multiple modalities such as titles, descriptions and aligned set of images. With over 36K automatically generated question-answer pairs, we design a set of comprehension and reasoning tasks that require joint understanding of images and text, capturing the temporal flow of events and making sense of procedural knowledge. Our preliminary results indicate that RecipeQA will serve as a challenging test bed and an ideal benchmark for evaluating machine comprehension systems. The data and leaderboard are available at http://hucvl.githu

In [9]:
#similar text - paraphrase plagiarism

model_text = 'Comprehending and deducing information from culinary instructions represents a promising avenue for research aimed at empowering artificial intelligence to decipher step-by-step text. In this study, we present CuisineInquiry, a database for the multifaceted understanding of cooking guidelines. It encompasses a substantial number of informative recipes featuring various elements such as headings, explanations, and a matched assortment of visuals. Utilizing an extensive set of automatically crafted question-answer pairings, we formulate a series of tasks focusing on understanding and logic that necessitate a combined interpretation of visuals and written content. This involves capturing the sequential progression of events and extracting meaning from procedural expertise. Our initial findings suggest that CuisineInquiry is poised to function as a demanding experimental platform.'

response = client.search(index='plagiarism-checker', size=1,
    knn={
        "field": "abstract_vector.predicted_value",
        "k": 9,
        "num_candidates": 974,
        "query_vector_builder": {
            "text_embedding": {
                "model_id": "sentence-transformers__all-mpnet-base-v2",
                "model_text": model_text
            }
        }
    }
)

for hit in response['hits']['hits']:
    score = hit['_score']
    title = hit['_source']['title']
    abstract = hit['_source']['abstract']
    openai = hit['_source']['openai-detector']['predicted_value']
    url = hit['_source']['url']

    if score > 0.9:
        print(f"\nHigh similarity detected! This might be plagiarism.")
        print(f"\nMost similar document: '{title}'\n\nAbstract: {abstract}\n\nurl: {url}\n\nScore:{score}\n")

        if openai == 'Fake':
            print("This document may have been created by AI.\n")

    elif score < 0.7:
        print(f"\nLow similarity detected. This might not be plagiarism.")

        if openai == 'Fake':
            print("This document may have been created by AI.\n")

    else:
        print(f"\nModerate similarity detected.")
        print(f"\nMost similar document: '{title}'\n\nAbstract: {abstract}\n\nurl: {url}\n\nScore:{score}\n")

        if openai == 'Fake':
            print("This document may have been created by AI.\n")

ml_client = MlClient(client)

model_id = 'roberta-base-openai-detector' #open ai text classification model

document = [
    {
        "text_field": model_text
    }
]

ml_response = ml_client.infer_trained_model(model_id=model_id, docs=document)

predicted_value = ml_response['inference_results'][0]['predicted_value']

if predicted_value == 'Fake':
    print("Note: The text query you entered may have been generated by AI.\n")



High similarity detected! This might be plagiarism.

Most similar document: 'RecipeQA: A Challenge Dataset for Multimodal Comprehension of Cooking Recipes'

Abstract: Understanding and reasoning about cooking recipes is a fruitful research direction towards enabling machines to interpret procedural text. In this work, we introduce RecipeQA, a dataset for multimodal comprehension of cooking recipes. It comprises of approximately 20K instructional recipes with multiple modalities such as titles, descriptions and aligned set of images. With over 36K automatically generated question-answer pairs, we design a set of comprehension and reasoning tasks that require joint understanding of images and text, capturing the temporal flow of events and making sense of procedural knowledge. Our preliminary results indicate that RecipeQA will serve as a challenging test bed and an ideal benchmark for evaluating machine comprehension systems. The data and leaderboard are available at http://hucvl.githu

In [12]:
#different text - not a plagiarism

model_text = 'Elasticsearch provides near real-time search and analytics for all types of data.'

response = client.search(index='plagiarism-checker', size=1, explain='true',
    knn={
        "field": "abstract_vector.predicted_value",
        "k": 9,
        "num_candidates": 974,
        "query_vector_builder": {
            "text_embedding": {
                "model_id": "sentence-transformers__all-mpnet-base-v2",
                "model_text": model_text
            }
        }
    }
)

for hit in response['hits']['hits']:
    score = hit['_score']
    title = hit['_source']['title']
    abstract = hit['_source']['abstract']
    openai = hit['_source']['openai-detector']['predicted_value']
    url = hit['_source']['url']

    if score > 0.9:
        print(f"\nHigh similarity detected! This might be plagiarism.")
        print(f"\nMost similar document: '{title}'\n\nAbstract: {abstract}\n\nurl: {url}\n\nScore:{score}\n")

        if openai == 'Fake':
            print("This document may have been created by AI.\n")

    elif score < 0.7:
        print(f"\nLow similarity detected. This might not be plagiarism. {score}")

        if openai == 'Fake':
            print("This document may have been created by AI.\n")

    else:
        print(f"\nModerate similarity detected.")
        print(f"\nMost similar document: '{title}'\n\nAbstract: {abstract}\n\nurl: {url}\n\nScore:{score}\n")

        if openai == 'Fake':
            print("This document may have been created by AI.\n")
      
ml_client = MlClient(client)

model_id = 'roberta-base-openai-detector' #open ai text classification model

document = [
    {
        "text_field": model_text
    }
]

ml_response = ml_client.infer_trained_model(model_id=model_id, docs=document)

predicted_value = ml_response['inference_results'][0]['predicted_value']

if predicted_value == 'Fake':
    print("Note: The text query you entered may have been generated by AI.\n")



Low similarity detected. This might not be plagiarism. 0.6814314
